In [1]:
import numpy as np # linear algebra
import pandas as pd
import os
import torch.utils.data as data
import torch
import gensim 
import pickle
from os import listdir
from os.path import join
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
args = {}
file_id = '1'
args['start_step'] = (int(file_id) - 1) *22500
args['end_step'] = int(file_id) * 22500
args['use_TextCNN'] = True
args['use_TextRCNN'] = True
print(os.listdir("../input"))
print(os.listdir("./"))

torch.cuda.is_available()

['fork-of-rcnn-model-4', 'tag-recommendation-system-list', 'train-validation-data-split', 'train-textcnn-best-model', 'hackerearth-deep-learning-challenge-4']
['.ipynb_checkpoints', '__notebook_source__.ipynb']


True

In [ ]:
def test_read_input(fold_dir):
    raw_data = pd.read_csv(fold_dir+'test.csv')
    logging.info("reading raw data...this may take a while")
    l = len(raw_data)
    article_list = []
    title_list = []
    id_list = []
    for i in range(l):
        line = raw_data['article'].iloc[i]
        title = raw_data['title'].iloc[i]
        id = raw_data['id'].iloc[i]
        if (i % 10000 == 0):
            logging.info("read {0} reviews".format(i))
        # do some pre-processing and return list of words for each article
        line = gensim.utils.simple_preprocess(line)
        title = gensim.utils.simple_preprocess(title)
        article_list.append(line)
        title_list.append(title)
        id_list.append(id)
    return id_list, title_list, article_list

test_id_list, test_title_list, test_article_list = test_read_input('../input/hackerearth-deep-learning-challenge-4/')

In [ ]:
def read_vocab(fold_dir):        
    f = open(fold_dir + 'label_vocab.pickle','rb')
    args['label_vocab'] = pickle.load(f)
    f.close()
    f = open(fold_dir + 'title_vocab.pickle','rb')
    args['title_vocab'] = pickle.load(f)
    f.close()
    f = open(fold_dir + 'article_vocab.pickle','rb')
    args['article_vocab'] = pickle.load(f)
    args['label_dim'] = len(args['label_vocab'])
    f.close()
    f = open(fold_dir + 'unknow_word_embedding.pickle','rb')
    args['unknow_for_article_embedding'],  args['unknow_for_title_embedding'] = pickle.load(f)
    f.close()
    return args

args = read_vocab('../input/tag-recommendation-system-list/')

In [ ]:
class Alphabet(dict):
    def __init__(self, start_id=1):
        self.fid = start_id
    def add(self, item):
        idx = self.get(item, None)
        if idx is None:
            idx = self.fid
            self[item] = idx
            self.fid += 1
        return idx
    def dump(self, fname):
        with open(fname, 'w') as out:
            for k in sorted(self.keys()):
                out.write("{}\t{}\n".format(k, self[k]))

def parse_embed(embed_dict,unknow_word):
    alphabet, embed_mat = Alphabet(), []
    vocab_size, embed_dim = len(embed_dict), len(embed_dict[list(embed_dict.keys())[0]])
    unknown_word_idx = 0
    embed_mat.append(unknow_word)
    for word in embed_dict:
        alphabet.add(word)
        embed_mat.append(embed_dict[word])
    dummy_word_idx = alphabet.fid
    embed_mat.append(np.zeros(embed_dim))
    return alphabet, embed_mat, unknown_word_idx, dummy_word_idx

test_title_alphabet, test_title_embed_mat, test_unknown_title_idx, test_dummy_title_idx = parse_embed(args['title_vocab'],args['unknow_for_title_embedding'])
test_article_alphabet, test_article_embed_mat, test_unknown_article_idx, test_dummy_article_idx = parse_embed(args['article_vocab'], args['unknow_for_article_embedding'])

In [ ]:
len_aritle = 500 # mean 176.22896799025133
len_title = 15 # mean 8.178064471860113

def pad_list(alphabet, embed_mat, unknown_idx, dummy_idx, list_file, length):
    result = []
    for l in list_file:
        if len(l) > length:
            l = l[len(l) - length:]
        else:
            l = [0] * (length - len(l)) + l
        tmp = []
        for i in l:
            if i == 0:
                tmp.append(embed_mat[dummy_idx])
            elif i not in alphabet:
                tmp.append(embed_mat[unknown_idx])
            elif i in alphabet:
                tmp.append(embed_mat[alphabet[i]])
        result.append(tmp)
    return result

args['test_title_embedding_pad'] = pad_list(test_title_alphabet, test_title_embed_mat, test_unknown_title_idx, test_dummy_title_idx, test_title_list, len_title)

del test_title_alphabet, test_title_embed_mat, test_unknown_title_idx, test_dummy_title_idx, test_title_list

args['test_article_embedding_pad'] = pad_list(test_article_alphabet, test_article_embed_mat, test_unknown_article_idx, test_dummy_article_idx, test_article_list, len_aritle)

del test_article_alphabet, test_article_embed_mat, test_unknown_article_idx, test_dummy_article_idx, test_article_list

In [ ]:
class TestDatasetFromFile(data.Dataset):
    def __init__(self, args, input_transform=True, start = 0, end = 0):
        super(TestDatasetFromFile, self).__init__()
        self.test_article_embedding_data = args['test_article_embedding_pad'][start:end]
        self.test_title_embedding_data = args['test_title_embedding_pad'][start:end]
        self.label_vocab = args['label_vocab']
        self.input_transform = input_transform

    # 在__getitem__中加载图片，并且将传入的transformation操作运用到
    # 加载的图片中。 `input = self.input_transforms(input)`
    # 这里的 self.input_transforms就是传入的"类的实例"，由于类是callable的
    # 所以可以 "类的实例（参数）"这样调用。在上一篇博客说到了这个。
    def __getitem__(self, index):
        article_input = self.test_article_embedding_data[index]
        title_input = self.test_title_embedding_data[index]
        if self.input_transform:
            input = [torch.FloatTensor(article_input),torch.FloatTensor(title_input)]
        else:
            input = torch.FloatTensor(article_input)
        return input

    def __len__(self):
        return len(self.test_title_embedding_data)


In [ ]:
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module

    def forward(self, x):
        if len(x.size()) <= 2:
            return self.module(x)
        n, t = x.size(0), x.size(1)
        # merge batch and seq dimensions
        x_reshape = x.contiguous().view(t * n, x.size(2))
        y = self.module(x_reshape)
        # we have to reshape Y
        y = y.contiguous().view(n, t, y.size()[1])
        return y

class K_MaxPooling(nn.Module):
    def __init__(self, k):
        super(K_MaxPooling, self).__init__()
        self.k = k
        
    def forward(self, x, dim):
        index = x.topk(self.k, dim=dim)[1].sort(dim=dim)[0]
        return x.gather(dim, index)

class textCNN(nn.Module):
    def __init__(self,args):
        super(textCNN, self).__init__()
        article_dim = args['article_dim'] #500
        title_dim = args['title_dim'] #15
        label_dim = args['label_dim'] # 36321
        train_size = args['train_size']
        embed_dim = args['embed_dim'] # 300
        # self.embeding = nn.Embedding(vocb_size, dim,_weight=embedding_matrix)

        D = embed_dim
        C = label_dim
        Ci = 1
        Co = 300#opt['kernel_num']
        #Ks = opt['kernel_sizes']
        Ks1 = [1,2,3,4,5]
        Ks2 = [3,4,5,6,7]
        
        self.tdfc1 = nn.Linear(D, 300)#512)
        self.td1 = TimeDistributed(self.tdfc1)
        self.tdbn1 = nn.BatchNorm2d(1)
        
        self.tdfc2 = nn.Linear(D, 300)#512)
        self.td2 = TimeDistributed(self.tdfc2)
        self.tdbn2 = nn.BatchNorm2d(1)

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, 290)) for K in Ks1])
        self.convbn1 = nn.ModuleList([nn.BatchNorm2d(Co) for i in range(len(Ks1))])
        self.convs2 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, 290)) for K in Ks2])
        self.convbn2 = nn.ModuleList([nn.BatchNorm2d(Co) for i in range(len(Ks2))])
        
        self.kmax_pooling = K_MaxPooling(5)
    
        self.fc1 = nn.Linear(33000, C)#4096)
        # self.fc1 = nn.Linear(115200, C)#4096)
        self.bn1 = nn.BatchNorm1d(C)#4096)
        self.fc2 = nn.Linear(C, C)#4096, C)

    def forward(self, x, y):
        batch_size = x.size(0)
        x = x.detach()
        x = F.relu(self.tdbn1(self.td1(x).unsqueeze(1)))
        y = y.detach()
        y = F.relu(self.tdbn2(self.td2(y).unsqueeze(1)))
        x = [F.relu(self.convbn1[i](conv(x))).squeeze(3) for i, conv in enumerate(self.convs1)]
        #print(x)
        # x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = [self.kmax_pooling(i, 2).mean(2).squeeze(1) for i in x]
        x = torch.cat(x, 1)
        
        y = [F.relu(self.convbn2[i](conv(y))).squeeze(3) for i, conv in enumerate(self.convs2)]
        # y = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in y]
        y = [self.kmax_pooling(i, 2).mean(2).squeeze(1) for i in y]
        
        y = torch.cat(y, 1)
        x = torch.cat((x, y), 1)
        x = x.view(batch_size, -1)
        x = F.relu(self.bn1(self.fc1(x)))
        logit = self.fc2(x)
        return logit

In [ ]:
class RCNN(nn.Module):
    def __init__(self,args):
        super(RCNN, self).__init__()
        article_dim = args['article_dim'] #500
        title_dim = args['title_dim'] #15
        label_dim = args['label_dim'] # 36321
        train_size = args['train_size']
        embed_dim = args['embed_dim'] # 300
        # self.embeding = nn.Embedding(vocb_size, dim,_weight=embedding_matrix)
        D = embed_dim
        C = label_dim
        self.num_layer = 1

        self.tdfc1 = nn.Linear(D, 300)
        self.td1 = TimeDistributed(self.tdfc1)
        self.tdbn1 = nn.BatchNorm2d(1)

        self.tdfc2 = nn.Linear(D, 300)
        self.td2 = TimeDistributed(self.tdfc2)
        self.tdbn2 = nn.BatchNorm2d(1)

        self.lstm1 = nn.LSTM(input_size = 300, hidden_size = 300, num_layers = self.num_layer, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size = 300, hidden_size = 300, num_layers = self.num_layer, batch_first=True, bidirectional=True)

        self.conv1 = nn.Conv2d(1, 1024 * 6, (3, 300+300+300))
        # self.conv1 = nn.Conv2d(1, 512, (1, 256+256+D))
        self.convbn1 = nn.BatchNorm2d(1024 * 6)
        # self.convbn1 = nn.BatchNorm2d(512)
        self.conv2 = nn.Conv2d(1, 1024 * 6, (3, 300+300+300))
        # self.conv2 = nn.Conv2d(1, 512, (1, 256+256+D))
        self.convbn2 = nn.BatchNorm2d(1024 * 6)
        # self.convbn2 = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(1024 * 6 * 2, C)
       	# self.fc = nn.Linear(1024, C)
        self.bn1 = nn.BatchNorm1d(C)#4096)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(C, C)#4096, C)   
        
    def forward(self, x, y):
        batch_size = x.size(0)
        x = x.detach()
        x = F.relu(self.tdbn1(self.td1(x).unsqueeze(1))).squeeze(1)
        y = y.detach()
        y = F.relu(self.tdbn2(self.td2(y).unsqueeze(1))).squeeze(1)

        h0_1 = torch.randn(self.num_layer * 2, batch_size, 300)
        c0_1 = torch.randn(self.num_layer * 2, batch_size, 300)
        # h0_1 = Variable(torch.randn(2, batch_size, self.D))
        # c0_1 = Variable(torch.randn(2, batch_size, self.D))
        h0_1 = h0_1.cuda()
        c0_1 = c0_1.cuda()
        o1, _ = self.lstm1(x, (h0_1, c0_1)) # (h0_1, c0_1) 
        x = torch.cat((x, o1), 2)
        x = F.relu(self.convbn1(self.conv1(x.unsqueeze(1))).squeeze(3))
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        
        # y = self.get_context_embedding(y, 2)
        h0_2 = Variable(torch.randn(self.num_layer * 2, batch_size, 300))
        c0_2 = Variable(torch.randn(self.num_layer * 2, batch_size, 300))
        # h0_2 = Variable(torch.randn(2, batch_size, self.D))
        # c0_2 = Variable(torch.randn(2, batch_size, self.D))
        h0_2 = h0_2.cuda()
        c0_2 = c0_2.cuda()
        o2, _ = self.lstm2(y, (h0_2, c0_2))
        y = torch.cat((y, o2), 2)
        
        y = F.relu(self.convbn2(self.conv2(y.unsqueeze(1))).squeeze(3))
        y = F.max_pool1d(y, y.size(2)).squeeze(2)
        
        x = torch.cat((x, y), 1)
        x = x.view(batch_size, -1)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        logit = self.fc2(x)
        
        return logit

In [ ]:
if args['use_TextCNN']:
    args['TextCNN_model'] = torch.load('../input/train-textcnn-best-model/TextCNN_model.pkl',map_location=lambda storage, loc: storage)
    args['TextCNN_model'].cuda()

In [ ]:
if args['use_TextRCNN']:
    args['TextRCNN_model'] = torch.load('../input/fork-of-rcnn-model-4/TextRCNN_model.pkl',map_location=lambda storage, loc: storage)
    args['TextRCNN_model'].cuda()

In [ ]:
test_dataset = torch.utils.data.DataLoader(
         TestDatasetFromFile(args, input_transform=True, start = args['start_step'], end = args['end_step']), 
         batch_size= 15, shuffle= False)

In [ ]:
def predict(args, test_dataset):
    label_dim = args['label_dim']
    if args['use_TextRCNN']:
        args['TextRCNN_model'].eval()
    if args['use_TextCNN']:
        args['TextCNN_model'].eval()
    
    #predict_label_list_CNN = []
    #predict_label_list_RCNN = []
    predict_label_list_all = []
    for step, data in enumerate(test_dataset):
        if step % 500 == 0:
            logging.info('Training step {} finished!'.format(step))
        train_x = data
        article_x, title_x = train_x[0], train_x[1]
        article_x, title_x = Variable(article_x), Variable(title_x)
        batch_size = article_x.size(0)
        article_x, title_x = article_x.cuda(), title_x.cuda()
        if args['use_TextRCNN'] and args['use_TextCNN']:
            output1 = args['TextRCNN_model'](article_x, title_x)
            output2 = args['TextCNN_model'](article_x, title_x)
            output = output1 + output2
        elif args['use_TextRCNN']:
            output = args['TextRCNN_model'](article_x, title_x)
        elif args['use_TextCNN']:
            output = args['TextCNN_model'](article_x, title_x)
        predict_label_list_all.extend([list(i) for i in np.array(output.cpu().data.numpy())])
        #predict_label_list_RCNN.extend([list(i) for i in np.array(output.cpu().data.numpy())])
        #predict_label_list_CNN.extend([list(i) for i in np.array(output.cpu().data.numpy())])
    logging.info('Training step {} finished!'.format(step))
    predict_label_list_all = np.array(predict_label_list_all)
    # predict_label_list_RCNN = np.array(predict_label_list_RCNN)
    # predict_label_list_CNN = np.array(predict_label_list_CNN)
    return predict_label_list_all #, predict_label_list_RCNN, predict_label_list_CNN

def threshold(predict_label_list,args):
    label_dim = args['label_dim']
    predict_y = []
    for observation in predict_label_list:
        labeled = observation > 0
        sum_labeled = sum(labeled)
        if 0 < sum_labeled <= 5:
            tmp = list(labeled)
        elif sum_labeled > 5:
            tmp = [0]*label_dim
            for i in np.argpartition(observation, -5)[-5:]:
                tmp[i] = 1
        elif sum_labeled == 0:
            tmp = [0]*label_dim
            for i in np.argpartition(observation, -3)[-3:]:
                tmp[i] = 1
        predict_y.append(tmp)
    predict_y = np.array(predict_y)
    return predict_y


In [ ]:
from datetime import datetime 
a=datetime.now()
predict_label_list_all = predict(args, test_dataset)
pred_y_all = threshold(predict_label_list_all,args)
#pred_y_RCNN = threshold(predict_label_list_RCNN,args)
#pred_y_CNN = threshold(predict_label_list_CNN,args)
b=datetime.now()
(b-a).seconds

In [ ]:
f = open('pred_y'+file_id+'_all.pickle','wb')
pickle.dump(pred_y_all,f)
f.close()
'''
f = open('pred_y'+file_id+'_RCNN.pickle','wb')
pickle.dump(pred_y_RCNN,f)
f.close()
f = open('pred_y'+file_id+'_CNN.pickle','wb')
pickle.dump(pred_y_CNN,f)
f.close()
'''

In [ ]:
'''
def create_predict_file(args,test_id_list,pred_y):
    args['id_to_label'] = {args['label_vocab'][i] : i for i in args['label_vocab']}
    f = open('predict.csv','w')
    f.write('id,tags\n')
    for line_id in range(len(pred_y)):
        y = pred_y[line_id]
        label_y = []
        for i in range(len(y)):
            if y[i] == 1:
                label_y.append(args['id_to_label'][i])
        label_str='|'.join(label_y)
        f.write('%s,%s\n' % (test_id_list[line_id], label_str))
    f.close()
    return

create_predict_file(args,test_id_list,pred_y)
'''

In [ ]:
#torch.save(model.cpu(), 'TextCNN_model.pkl')  # 保存整个网络
# torch.save(model.state_dict(), 'TextCNN_model_para.pkl')   # 只保存网络中的参数 (速度快, 占内存少)

In [ ]:
#torch.save(model.cpu().state_dict(), 'TextCNN_model_para.pkl')

In [ ]:
# TextCNN_model = torch.load('TextCNN_model.pkl',map_location=lambda storage, loc: storage)
# TextCNN_model_para = textCNN(args)
# TextCNN_model_para.load_state_dict(torch.load('TextCNN_model_para.pkl',map_location=lambda storage, loc: storage))